# Ordered Logit/Probit: Ordinal Dependent Variables

**Tutorial Series**: Discrete Choice Econometrics with PanelBox

**Notebook**: 07 - Ordered Models

**Author**: PanelBox Contributors

**Date**: 2026-02-17

**Estimated Duration**: 75 minutes

**Difficulty Level**: Intermediate-Advanced

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Distinguish ordinal from nominal (multinomial) dependent variables
2. Specify the latent variable framework with cutpoints (thresholds)
3. Estimate Ordered Logit and Ordered Probit using PanelBox
4. Interpret cutpoints and their relationship to category boundaries
5. Test the parallel regression (proportional odds) assumption
6. Calculate category-specific marginal effects
7. Understand Random Effects Ordered models for panel data

---

## Table of Contents

1. [Ordinal vs Multinomial](#section1)
2. [Latent Variable Framework](#section2)
3. [Cutpoints Interpretation](#section3)
4. [Estimation with PanelBox](#section4)
5. [Parallel Regression Assumption](#section5)
6. [Predicted Probabilities per Category](#section6)
7. [Category-Specific Marginal Effects](#section7)
8. [Random Effects Ordered Logit](#section8)
9. [Application — Credit Rating Analysis](#section9)
10. [Exercises](#exercises)

---

## Prerequisites

- **Required**: Notebook 01 (Binary Choice Introduction)
- **Recommended**: Notebook 06 (Multinomial Logit) for comparison
- **Conceptual**: Latent variables, cumulative distributions, ordered categories
- **Technical**: Understanding of CDFs and thresholds

## Setup

Import all required libraries and configure the environment.

In [ ]:
# Standard library imports
import warnings
from pathlib import Path

# Data manipulation and numerical computing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Statistical functions
from scipy.stats import norm, chi2
from scipy.special import expit

# PanelBox models
from panelbox.models.discrete.ordered import (
    OrderedLogit, OrderedProbit, RandomEffectsOrderedLogit
)

# Configuration
warnings.filterwarnings('ignore')
np.random.seed(42)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

# Matplotlib configuration
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Paths
DATA_DIR = Path("..") / "data"
OUTPUT_DIR = Path("..") / "outputs"
FIG_DIR = OUTPUT_DIR / "figures"
TABLE_DIR = OUTPUT_DIR / "tables"
REPORT_DIR = OUTPUT_DIR / "reports"

# Create output directories if needed
FIG_DIR.mkdir(parents=True, exist_ok=True)
TABLE_DIR.mkdir(parents=True, exist_ok=True)
REPORT_DIR.mkdir(parents=True, exist_ok=True)

# Rating labels for readability
RATING_LABELS = {0: 'Poor', 1: 'Fair', 2: 'Good', 3: 'Excellent'}
RATING_COLORS = ['#e74c3c', '#f39c12', '#3498db', '#2ecc71']

print("All libraries imported successfully")
print(f"Working directory: {Path.cwd()}")

<a id='section1'></a>

---

# Section 1: Ordinal vs Multinomial (20 min)

## 1.1 What Are Ordinal Data?

Ordinal data have categories with a **natural ordering** but **unknown distances** between categories:

- Credit rating: poor < fair < good < excellent
- Customer satisfaction: 1 < 2 < 3 < 4 < 5
- Education level: primary < secondary < tertiary
- Health status: poor < fair < good < very good < excellent

## 1.2 Why Not Multinomial Logit?

Multinomial Logit (Notebook 06) treats all categories as **unordered**:
- **Ignores ordering information** inherent in the data
- Requires $(J-1) \times K$ parameters vs $K + (J-1)$ for ordered models
- Less efficient: wastes degrees of freedom

## 1.3 Why Not OLS?

Treating ordinal categories as continuous (OLS regression) has problems:
- Assumes **equal distances** between categories (is the gap from "poor" to "fair" the same as "good" to "excellent"?)
- Predicted values can **fall outside the valid range** (e.g., predicted rating of -0.5 or 4.3)
- Heteroskedastic errors by construction

## 1.4 Ordered Models: Best of Both Worlds

Ordered Logit/Probit models:
- **Preserve the ordering** (poor < fair < good < excellent)
- Allow **flexible category boundaries** (distances estimated from data)
- Predictions are **proper probabilities** that sum to 1
- Require only $K + (J-1)$ parameters

## 1.5 Load the Data

In [ ]:
# Load credit rating panel data
data = pd.read_csv(DATA_DIR / "credit_rating.csv")

print("Dataset loaded successfully!")
print(f"\nShape: {data.shape}")
print(f"Number of firms: {data['id'].nunique()}")
print(f"Number of periods: {data['year'].nunique()}")
print(f"Years: {data['year'].min()} - {data['year'].max()}")
print(f"\nFirst 10 rows:")
data.head(10)

In [ ]:
# Rating distribution
print("=== Credit Rating Distribution ===")
rating_dist = data['rating'].value_counts().sort_index()
rating_prop = data['rating'].value_counts(normalize=True).sort_index()

for code, label in RATING_LABELS.items():
    print(f"  {code} ({label:10s}): {rating_dist[code]:5d} obs  ({rating_prop[code]:.1%})")

print(f"\nTotal observations: {len(data)}")

In [ ]:
# Summary statistics by rating category
print("=== Firm Characteristics by Credit Rating ===")
summary = data.groupby('rating')[['income', 'debt_ratio', 'age', 'size', 'profitability']].mean()
summary.index = [RATING_LABELS[r] for r in summary.index]
print(summary.round(3))

print("\nKey patterns:")
print("  - Higher income -> better ratings")
print("  - Lower debt -> better ratings")
print("  - Larger and more profitable firms tend to have better ratings")

In [ ]:
# Visualize rating distribution and characteristics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Rating distribution
labels = [RATING_LABELS[i] for i in range(4)]
bars = axes[0, 0].bar(labels, rating_dist.values, color=RATING_COLORS, alpha=0.8, edgecolor='black')
for bar, count in zip(bars, rating_dist.values):
    axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
                    f'{count}', ha='center', va='bottom', fontsize=11)
axes[0, 0].set_title('Credit Rating Distribution', fontweight='bold')
axes[0, 0].set_ylabel('Count')
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Income by rating
for code, label in RATING_LABELS.items():
    subset = data[data['rating'] == code]['income']
    axes[0, 1].hist(subset, bins=25, alpha=0.5, label=label, color=RATING_COLORS[code])
axes[0, 1].set_title('Income Distribution by Rating', fontweight='bold')
axes[0, 1].set_xlabel('Log Firm Income')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Debt ratio by rating
rating_data = [data[data['rating'] == c]['debt_ratio'] for c in range(4)]
bp = axes[1, 0].boxplot(rating_data, labels=labels, patch_artist=True, notch=True)
for patch, color in zip(bp['boxes'], RATING_COLORS):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)
axes[1, 0].set_title('Debt Ratio by Rating', fontweight='bold')
axes[1, 0].set_ylabel('Debt-to-Assets Ratio')
axes[1, 0].grid(True, alpha=0.3)

# 4. Rating transitions over time
rating_by_year = data.groupby('year')['rating'].value_counts(normalize=True).unstack(fill_value=0)
rating_by_year.columns = [RATING_LABELS[c] for c in rating_by_year.columns]
rating_by_year.plot(kind='bar', stacked=True, ax=axes[1, 1],
                    color=RATING_COLORS, alpha=0.8, edgecolor='black')
axes[1, 1].set_title('Rating Distribution Over Time', fontweight='bold')
axes[1, 1].set_ylabel('Proportion')
axes[1, 1].set_xlabel('Year')
axes[1, 1].tick_params(axis='x', rotation=0)
axes[1, 1].legend(title='Rating')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Credit Rating: Exploratory Analysis', fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig(FIG_DIR / '07_data_exploration.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved to outputs/figures/07_data_exploration.png")

<a id='section2'></a>

---

# Section 2: Latent Variable Framework (25 min)

## 2.1 The Latent Variable

Ordered models posit an **unobserved** continuous latent variable $y^*$:

$$y^*_{it} = X_{it}'\beta + \varepsilon_{it}$$

- $y^*$ represents the firm's underlying **credit quality** (continuous, unbounded)
- We observe only the **discrete rating** category, not $y^*$ itself

## 2.2 Cutpoints Map Latent Variable to Categories

**Cutpoints** (thresholds) $\kappa_0, \kappa_1, \kappa_2$ define the boundaries:

$$
y_{it} = \begin{cases}
0 \text{ (poor)} & \text{if } y^* \leq \kappa_0 \\
1 \text{ (fair)} & \text{if } \kappa_0 < y^* \leq \kappa_1 \\
2 \text{ (good)} & \text{if } \kappa_1 < y^* \leq \kappa_2 \\
3 \text{ (excellent)} & \text{if } y^* > \kappa_2
\end{cases}
$$

**Key**: $J$ categories require $J-1$ cutpoints (here: 4 categories, 3 cutpoints).

## 2.3 Category Probabilities

$$P(y=j \mid X) = F(\kappa_j - X'\beta) - F(\kappa_{j-1} - X'\beta)$$

where:
- $F = \Lambda$ (Logistic CDF) for **Ordered Logit**
- $F = \Phi$ (Normal CDF) for **Ordered Probit**

## 2.4 Parameters to Estimate

- $\beta$: coefficient vector ($K$ parameters) — common across all thresholds
- $\kappa_0, \kappa_1, \ldots, \kappa_{J-2}$: cutpoints ($J-1$ parameters)
- Total: $K + (J-1)$ parameters

## 2.5 Visualization: Latent Variable and Cutpoints

In [ ]:
# Visualize the latent variable framework
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# --- Panel A: Latent variable density with cutpoints ---
ax = axes[0]
x = np.linspace(-5, 7, 500)

# Example: logistic density (standard)
logistic_pdf = np.exp(-x) / (1 + np.exp(-x))**2

# Example cutpoints
kappa = [-0.8, 0.4, 1.6]

# Fill regions
regions = [
    (-5, kappa[0], RATING_COLORS[0], 'Poor (y=0)'),
    (kappa[0], kappa[1], RATING_COLORS[1], 'Fair (y=1)'),
    (kappa[1], kappa[2], RATING_COLORS[2], 'Good (y=2)'),
    (kappa[2], 7, RATING_COLORS[3], 'Excellent (y=3)'),
]

for lo, hi, color, label in regions:
    mask = (x >= lo) & (x <= hi)
    ax.fill_between(x[mask], logistic_pdf[mask], alpha=0.3, color=color, label=label)

ax.plot(x, logistic_pdf, 'k-', linewidth=2, label='f(y*)')

# Cutpoint lines
for j, k in enumerate(kappa):
    ax.axvline(x=k, color='black', linestyle='--', linewidth=1.5, alpha=0.7)
    ax.text(k, ax.get_ylim()[1] * 0.95, f'$\\kappa_{j}$ = {k:.1f}',
            ha='center', va='top', fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='black', alpha=0.8))

ax.set_xlabel('Latent variable y* (credit quality)', fontsize=12)
ax.set_ylabel('Density f(y*)', fontsize=12)
ax.set_title('Panel A: Latent Variable Density with Cutpoints', fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# --- Panel B: Cumulative probabilities ---
ax = axes[1]

# Show P(y <= j) = F(kappa_j - X'beta) for different X'beta values
xb_values = np.linspace(-3, 5, 200)

for j, k in enumerate(kappa):
    cum_prob = expit(k - xb_values)  # F(kappa_j - X'beta)
    ax.plot(xb_values, cum_prob, linewidth=2, color=RATING_COLORS[j],
            label=f'P(y $\\leq$ {j}) = F($\\kappa_{j}$ - X\'$\\beta$)')

ax.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
ax.set_xlabel("X'$\\beta$ (linear predictor)", fontsize=12)
ax.set_ylabel('Cumulative Probability', fontsize=12)
ax.set_title('Panel B: Cumulative Probability Curves', fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_ylim(-0.05, 1.05)

plt.suptitle('Ordered Choice: Latent Variable Framework', fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig(FIG_DIR / '07_latent_variable_framework.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved to outputs/figures/07_latent_variable_framework.png")

<a id='section3'></a>

---

# Section 3: Cutpoints Interpretation (20 min)

## 3.1 Key Properties of Cutpoints

1. **Ordering constraint**: $\kappa_0 < \kappa_1 < \ldots < \kappa_{J-2}$ (enforced via exponential reparameterization)
2. **Estimated simultaneously** with $\beta$ via maximum likelihood
3. **Gaps between cutpoints** reflect category spacing:
   - Large gap $\kappa_j - \kappa_{j-1}$ $\rightarrow$ many observations in category $j$
   - Small gap $\rightarrow$ few observations in category $j$
4. **Not interpretable in isolation** — they depend on the scale (no intercept in $X'\beta$)

## 3.2 Reparameterization

PanelBox ensures ordering via:
$$\kappa_0 = \gamma_0, \quad \kappa_j = \kappa_{j-1} + \exp(\gamma_j) \quad \text{for } j > 0$$

Since $\exp(\gamma_j) > 0$, this guarantees $\kappa_j > \kappa_{j-1}$.

## 3.3 Example: What Cutpoints Tell Us

If estimated cutpoints are $\hat{\kappa}_0 = -0.8$, $\hat{\kappa}_1 = 0.4$, $\hat{\kappa}_2 = 1.6$:
- Gap between Poor/Fair boundary and Fair/Good boundary: $0.4 - (-0.8) = 1.2$
- Gap between Fair/Good and Good/Excellent: $1.6 - 0.4 = 1.2$
- Equal gaps suggest roughly equal difficulty in transitioning between adjacent categories

We'll see the actual estimated cutpoints after fitting the model.

<a id='section4'></a>

---

# Section 4: Estimation with PanelBox (15 min)

## 4.1 Prepare the Data

In [ ]:
# Prepare variables
exog_vars = ['income', 'debt_ratio', 'age', 'size', 'profitability']
y = data['rating'].values
X = data[exog_vars].values
groups = data['id'].values
time = data['year'].values

print(f"Dependent variable (rating): {np.unique(y)}")
print(f"Number of covariates: {len(exog_vars)}")
print(f"Total observations: {len(y)}")
print(f"Number of categories: {len(np.unique(y))}")
print(f"Parameters to estimate: {len(exog_vars)} betas + {len(np.unique(y)) - 1} cutpoints = {len(exog_vars) + len(np.unique(y)) - 1}")

## 4.2 Ordered Logit

In [ ]:
# Estimate Ordered Logit
model_ologit = OrderedLogit(
    endog=y,
    exog=X,
    groups=groups,
    time=time,
    n_categories=4
)

# Store variable names for later use
model_ologit.exog_names = exog_vars

results_ologit = model_ologit.fit()

print("=" * 70)
print(" " * 15 + "ORDERED LOGIT: CREDIT RATINGS")
print("=" * 70)
print(results_ologit.summary())

## 4.3 Coefficient Interpretation

In ordered models, **$\beta_k > 0$** means:
- Higher $X_k$ $\rightarrow$ higher latent $y^*$ $\rightarrow$ higher probability of **better** rating
- But $\beta_k$ is **not** the marginal effect on any specific category probability

The sign tells us the **direction**: which end of the scale benefits from an increase in $X_k$.

In [ ]:
# Detailed interpretation
print("=== Coefficient Interpretation ===")
print("\nSign interpretation (effect on latent credit quality y*):")

K = len(exog_vars)
for k, var in enumerate(exog_vars):
    coef = results_ologit.beta[k]
    se = results_ologit.bse[k]
    z = coef / se if se > 0 else np.nan
    p = 2 * (1 - norm.cdf(abs(z))) if not np.isnan(z) else np.nan
    sig = '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.1 else ''

    direction = 'better rating' if coef > 0 else 'worse rating'
    print(f"  {var:15s}: beta = {coef:+.4f} {sig:3s}  -> Higher {var} -> {direction}")

print(f"\nCutpoints (ordered):")
for j, kappa in enumerate(results_ologit.cutpoints):
    print(f"  kappa_{j}: {kappa:.4f}")

print(f"\nCutpoint gaps:")
for j in range(len(results_ologit.cutpoints) - 1):
    gap = results_ologit.cutpoints[j+1] - results_ologit.cutpoints[j]
    print(f"  kappa_{j+1} - kappa_{j} = {gap:.4f}")

## 4.4 Ordered Probit (Comparison)

In [ ]:
# Estimate Ordered Probit
model_oprobit = OrderedProbit(
    endog=y,
    exog=X,
    groups=groups,
    time=time,
    n_categories=4
)
model_oprobit.exog_names = exog_vars

results_oprobit = model_oprobit.fit()

print("=" * 70)
print(" " * 15 + "ORDERED PROBIT: CREDIT RATINGS")
print("=" * 70)
print(results_oprobit.summary())

In [ ]:
# Compare Ordered Logit vs Ordered Probit
print("=== Ordered Logit vs Ordered Probit ===")
print(f"\n{'Variable':<15} {'Logit':>10} {'Probit':>10} {'Ratio':>10}")
print("-" * 50)
for k, var in enumerate(exog_vars):
    b_logit = results_ologit.beta[k]
    b_probit = results_oprobit.beta[k]
    ratio = b_logit / b_probit if abs(b_probit) > 1e-6 else np.nan
    print(f"{var:<15} {b_logit:>10.4f} {b_probit:>10.4f} {ratio:>10.2f}")

print(f"\nLog-likelihood:")
print(f"  Logit:  {results_ologit.llf:.2f}")
print(f"  Probit: {results_oprobit.llf:.2f}")

# Compute AIC/BIC
n_params = K + 3  # beta + cutpoints
n_obs = len(y)
aic_logit = -2 * results_ologit.llf + 2 * n_params
aic_probit = -2 * results_oprobit.llf + 2 * n_params
bic_logit = -2 * results_ologit.llf + np.log(n_obs) * n_params
bic_probit = -2 * results_oprobit.llf + np.log(n_obs) * n_params

print(f"\nAIC:  Logit = {aic_logit:.2f}, Probit = {aic_probit:.2f} -> {'Logit' if aic_logit < aic_probit else 'Probit'} preferred")
print(f"BIC:  Logit = {bic_logit:.2f}, Probit = {bic_probit:.2f} -> {'Logit' if bic_logit < bic_probit else 'Probit'} preferred")
print(f"\nNote: Logit/Probit coefficients differ by a scale factor of ~1.6-1.8.")
print(f"This is expected: logistic variance = pi^2/3, normal variance = 1.")

In [ ]:
# Visualize cutpoints on number line
fig, ax = plt.subplots(figsize=(14, 4))

cutpoints = results_ologit.cutpoints

# Number line
x_range = [cutpoints[0] - 2, cutpoints[-1] + 2]
ax.axhline(y=0, color='black', linewidth=2, zorder=1)

# Shade regions
boundaries = [x_range[0]] + list(cutpoints) + [x_range[1]]
for j in range(4):
    ax.axvspan(boundaries[j], boundaries[j+1], alpha=0.2, color=RATING_COLORS[j])
    mid = (boundaries[j] + boundaries[j+1]) / 2
    ax.text(mid, 0.3, f'{RATING_LABELS[j]}\n(y={j})', ha='center', va='bottom',
            fontsize=12, fontweight='bold', color=RATING_COLORS[j])

# Cutpoint markers
for j, k in enumerate(cutpoints):
    ax.plot(k, 0, 'ko', markersize=12, zorder=5)
    ax.annotate(f'$\\hat{{\\kappa}}_{j}$ = {k:.2f}', xy=(k, 0), xytext=(k, -0.5),
                ha='center', fontsize=11, fontweight='bold',
                arrowprops=dict(arrowstyle='->', color='black'),
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', edgecolor='black'))

ax.set_xlim(x_range)
ax.set_ylim(-1.0, 0.8)
ax.set_xlabel('Latent variable y* (credit quality)', fontsize=12)
ax.set_title('Estimated Cutpoints from Ordered Logit', fontweight='bold', fontsize=14)
ax.set_yticks([])
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(FIG_DIR / '07_cutpoints_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved to outputs/figures/07_cutpoints_visualization.png")

<a id='section5'></a>

---

# Section 5: Parallel Regression Assumption (20 min)

## 5.1 The Proportional Odds Assumption

The Ordered Logit assumes that $\beta$ is **common across all thresholds**:

$$\log \frac{P(y \leq j)}{P(y > j)} = \kappa_j - X'\beta \quad \text{for all } j$$

The effect of $X_k$ on the log-odds of $y \leq j$ vs $y > j$ is **constant** for all $j$.

## 5.2 Violation

If income matters more for the "good $\rightarrow$ excellent" transition than for "poor $\rightarrow$ fair", the parallel regression assumption is violated.

## 5.3 Brant Test

**Procedure**:
1. Estimate $J-1$ separate binary logits (dichotomize at each threshold)
2. Compare $\beta$ across these models
3. $H_0$: $\beta$ are equal (parallel regression holds)
4. Test statistic: $\chi^2$ with $(J-2) \times K$ degrees of freedom

## 5.4 Implementation

In [ ]:
# Brant test for proportional odds
# Estimate J-1 separate binary logits using statsmodels
import statsmodels.api as sm

n_categories = 4
n_thresholds = n_categories - 1  # 3 binary models

binary_betas = []
binary_vcovs = []

print("=== Separate Binary Logit Models ===")
print(f"\nDichotomizing at each threshold:\n")

# Add constant to X for binary logit models
X_with_const = sm.add_constant(X)

for j in range(n_thresholds):
    # Create binary outcome: y <= j vs y > j
    y_binary = (y > j).astype(int)

    # Estimate binary logit via statsmodels
    logit_model = sm.Logit(y_binary, X_with_const)
    res_binary = logit_model.fit(disp=False)

    # Extract coefficients (excluding intercept)
    beta_j = res_binary.params[1:]  # Skip intercept
    binary_betas.append(beta_j)

    # Extract covariance (excluding intercept rows/cols)
    vcov_j = res_binary.cov_params()[1:, 1:]
    binary_vcovs.append(vcov_j)

    print(f"  Binary Logit: P(rating > {j}) vs P(rating <= {j})")
    print(f"  N(y=1) = {y_binary.sum()}, N(y=0) = {(1-y_binary).sum()}")
    coefs_str = ', '.join(f"{var}={beta_j[k]:+.3f}" for k, var in enumerate(exog_vars))
    print(f"  Coefficients: {coefs_str}")
    print()

In [ ]:
# Compute Brant test statistic
print("=" * 70)
print(" " * 20 + "BRANT TEST")
print("=" * 70)
print("\nH0: Parallel regression (proportional odds) assumption holds")
print("H1: Coefficients differ across thresholds\n")

# Per-variable test: compare beta across J-1 binary models
brant_results = []
beta_matrix = np.array(binary_betas)  # (J-1) x K

for k, var in enumerate(exog_vars):
    # Get betas for variable k across all thresholds
    betas_k = beta_matrix[:, k]

    # Pairwise differences from the first binary model
    diffs = betas_k[1:] - betas_k[0]

    # Variance of differences (simplified: use diagonal elements)
    var_diffs = np.array([
        binary_vcovs[j+1][k, k] + binary_vcovs[0][k, k]
        for j in range(len(diffs))
    ])

    # Chi-squared statistic for this variable
    chi2_k = np.sum(diffs**2 / var_diffs)
    df_k = len(diffs)
    p_k = 1 - chi2.cdf(chi2_k, df_k)

    brant_results.append({
        'Variable': var,
        'chi2': chi2_k,
        'df': df_k,
        'p_value': p_k,
        'Conclusion': 'Reject' if p_k < 0.05 else 'Fail to reject'
    })

# Overall test
all_diffs = beta_matrix[1:] - beta_matrix[0]  # (J-2) x K
chi2_overall = 0
for j in range(all_diffs.shape[0]):
    vcov_diff = binary_vcovs[j+1] + binary_vcovs[0]
    try:
        chi2_overall += float(all_diffs[j] @ np.linalg.inv(vcov_diff) @ all_diffs[j])
    except np.linalg.LinAlgError:
        chi2_overall += float(np.sum(all_diffs[j]**2 / np.diag(vcov_diff)))

df_overall = (n_thresholds - 1) * K
p_overall = 1 - chi2.cdf(chi2_overall, df_overall)

# Display results
print(f"{'Variable':<15} {'chi2':>10} {'df':>5} {'p-value':>10} {'Conclusion':>15}")
print("-" * 60)
for res in brant_results:
    print(f"{res['Variable']:<15} {res['chi2']:>10.3f} {res['df']:>5d} {res['p_value']:>10.4f} {res['Conclusion']:>15}")
print("-" * 60)
print(f"{'Overall':<15} {chi2_overall:>10.3f} {df_overall:>5d} {p_overall:>10.4f} {'Reject' if p_overall < 0.05 else 'Fail to reject':>15}")

print(f"\nInterpretation:")
if p_overall >= 0.05:
    print(f"  The overall Brant test does not reject H0 (p = {p_overall:.4f}).")
    print(f"  The proportional odds assumption appears reasonable.")
else:
    print(f"  The overall Brant test rejects H0 (p = {p_overall:.4f}).")
    print(f"  Consider Generalized Ordered Logit or inspect per-variable results.")

In [ ]:
# Visualize: coefficient estimates from separate binary logits
fig, ax = plt.subplots(figsize=(14, 6))

x_pos = np.arange(len(exog_vars))
width = 0.25
threshold_colors = ['#e74c3c', '#f39c12', '#3498db']

for j in range(n_thresholds):
    offset = (j - 1) * width
    bars = ax.bar(x_pos + offset, binary_betas[j], width,
                  label=f'P(rating > {j})', color=threshold_colors[j],
                  alpha=0.8, edgecolor='black')

# Add ordered logit coefficients as reference line markers
for k in range(len(exog_vars)):
    ax.plot([x_pos[k] - 0.4, x_pos[k] + 0.4],
            [results_ologit.beta[k], results_ologit.beta[k]],
            'k--', linewidth=2, alpha=0.5)

ax.axhline(y=0, color='black', linewidth=0.8, alpha=0.5)
ax.set_xlabel('Variable')
ax.set_ylabel('Coefficient Estimate')
ax.set_title('Parallel Regression Test: Binary Logit Coefficients by Threshold\n'
             '(dashed line = Ordered Logit common coefficient)', fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(exog_vars)
ax.legend(title='Binary Model')
ax.grid(True, alpha=0.3, axis='y')

# Add custom legend entry for ordered logit reference
handles, labels_leg = ax.get_legend_handles_labels()
handles.append(plt.Line2D([0], [0], color='black', linestyle='--', linewidth=2, alpha=0.5))
labels_leg.append('Ordered Logit (pooled)')
ax.legend(handles, labels_leg, title='Model')

plt.tight_layout()
plt.savefig(FIG_DIR / '07_parallel_regression_test.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved to outputs/figures/07_parallel_regression_test.png")
print("\nIf bars within each variable are approximately equal,")
print("the parallel regression assumption holds.")

<a id='section6'></a>

---

# Section 6: Predicted Probabilities per Category (15 min)

## 6.1 Category Probabilities

For each observation, we compute:

$$P(y = j \mid X) = F(\kappa_j - X'\beta) - F(\kappa_{j-1} - X'\beta)$$

Key properties:
- $\sum_{j=0}^{J-1} P(y = j \mid X) = 1$
- Predicted class: $\hat{y} = \arg\max_j P(y=j \mid X)$

In [ ]:
# Predicted probabilities for all observations
probs = results_ologit.predict_proba()

print(f"Predicted probabilities shape: {probs.shape}")
print(f"  {probs.shape[0]} observations x {probs.shape[1]} categories\n")

# First 10 observations
prob_df = pd.DataFrame(probs, columns=[RATING_LABELS[j] for j in range(4)])
print("First 10 predicted probability vectors:")
print(prob_df.head(10).round(4))

# Verify rows sum to 1
row_sums = probs.sum(axis=1)
print(f"\nRow sums: min = {row_sums.min():.10f}, max = {row_sums.max():.10f}")
print(f"All sum to 1: {np.allclose(row_sums, 1.0)}")

In [ ]:
# Predictions for representative firm profiles
profiles = pd.DataFrame({
    'Profile': [
        'Healthy firm',
        'Average firm',
        'Distressed firm',
        'High growth'
    ],
    'income':        [12.0,  10.5,  9.0,   11.5],
    'debt_ratio':    [0.15,  0.40,  0.70,  0.30],
    'age':           [30,    20,    10,    5],
    'size':          [10.0,  8.0,   6.5,   9.0],
    'profitability': [0.15,  0.07,  0.01,  0.12]
})

X_profiles = profiles[exog_vars].values
probs_profiles = results_ologit.predict_proba(X_profiles)

print("=== Predicted Probabilities for Representative Firms ===")
for i, row in profiles.iterrows():
    print(f"\n{row['Profile']}:")
    print(f"  (income={row['income']}, debt={row['debt_ratio']}, age={row['age']}, "
          f"size={row['size']}, profit={row['profitability']})")
    for j in range(4):
        bar_len = int(probs_profiles[i, j] * 40)
        bar = '#' * bar_len + ' ' * (40 - bar_len)
        print(f"  P({RATING_LABELS[j]:10s}) = {probs_profiles[i, j]:.4f}  |{bar}|")

In [ ]:
# Stacked bar chart of predicted probabilities for representative profiles
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(profiles))
bottom = np.zeros(len(profiles))

for j in range(4):
    label = RATING_LABELS[j]
    values = probs_profiles[:, j]
    ax.bar(x, values, bottom=bottom, label=label,
           color=RATING_COLORS[j], alpha=0.8, edgecolor='black')

    # Add percentage labels
    for i, (v, b) in enumerate(zip(values, bottom)):
        if v > 0.04:
            ax.text(i, b + v/2, f'{v:.0%}', ha='center', va='center',
                    fontsize=9, fontweight='bold')
    bottom += values

ax.set_xlabel('Firm Profile')
ax.set_ylabel('Probability')
ax.set_title('Predicted Rating Probabilities for Representative Firms', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(profiles['Profile'], rotation=0)
ax.legend(title='Rating')
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIG_DIR / '07_predicted_probabilities.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved to outputs/figures/07_predicted_probabilities.png")

<a id='section7'></a>

---

# Section 7: Category-Specific Marginal Effects (25 min)

## 7.1 Why Marginal Effects Differ by Category

In ordered models, marginal effects are **category-specific**:

$$\frac{\partial P(y=j)}{\partial x_k} = \beta_k \times \left[ f(\kappa_{j-1} - X'\beta) - f(\kappa_j - X'\beta) \right]$$

## 7.2 Key Properties

1. **Sum to zero**: $\sum_j \frac{\partial P(y=j)}{\partial x_k} = 0$
2. **Extreme categories**: sign follows $\beta$ (if $\beta > 0$, P(highest) increases, P(lowest) decreases)
3. **Intermediate categories**: sign is **ambiguous** — can increase or decrease!

## 7.3 Average Marginal Effects

In [ ]:
# Compute category-specific Average Marginal Effects manually
# (Direct computation to ensure compatibility with the model's API)

beta = results_ologit.beta
cutpoints = results_ologit.cutpoints
cutpoints_ext = np.concatenate([[-np.inf], cutpoints, [np.inf]])

# Logistic PDF
def logistic_pdf(z):
    F = expit(z)
    return F * (1 - F)

n_obs = len(X)
n_cats = 4

# AME: average over all observations
ame_matrix = np.zeros((len(exog_vars), n_cats))

for i in range(n_obs):
    linear_pred = X[i] @ beta

    for j in range(n_cats):
        z_lower = cutpoints_ext[j] - linear_pred
        z_upper = cutpoints_ext[j + 1] - linear_pred

        pdf_lower = logistic_pdf(z_lower) if np.isfinite(z_lower) else 0
        pdf_upper = logistic_pdf(z_upper) if np.isfinite(z_upper) else 0

        for k in range(len(exog_vars)):
            ame_matrix[k, j] += beta[k] * (pdf_lower - pdf_upper)

ame_matrix /= n_obs

# Create DataFrame
ame_df = pd.DataFrame(
    ame_matrix,
    index=exog_vars,
    columns=[RATING_LABELS[j] for j in range(n_cats)]
)

print("=== Average Marginal Effects (AME) by Category ===")
print("\nEffect of a one-unit increase in each variable on P(rating=j)\n")
print(ame_df.round(4))

In [ ]:
# Verify sum-to-zero property
print("=== Sum-to-Zero Verification ===")
print("\nSum of AME across categories for each variable:")
for k, var in enumerate(exog_vars):
    row_sum = ame_matrix[k].sum()
    status = 'PASS' if abs(row_sum) < 1e-8 else 'CHECK'
    print(f"  {var:15s}: {row_sum:+.10f}  [{status}]")

print("\nThis confirms: probability mass shifts between categories, total remains 1.")

In [ ]:
# Heatmap of AME (categories x variables)
fig, ax = plt.subplots(figsize=(10, 5))

sns.heatmap(ame_df.T, annot=True, fmt='.4f', cmap='RdBu_r', center=0,
            ax=ax, linewidths=1,
            cbar_kws={'label': 'Average Marginal Effect'})
ax.set_title('Average Marginal Effects by Category\n'
             '(effect on P(rating=j) per unit change in variable)',
             fontweight='bold')
ax.set_ylabel('Rating Category')
ax.set_xlabel('Variable')

plt.tight_layout()
plt.savefig(FIG_DIR / '07_marginal_effects_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved to outputs/figures/07_marginal_effects_heatmap.png")

In [ ]:
# Key insight: intermediate category ambiguity
print("=== Key Insight: Intermediate Category Ambiguity ===")
print("\nFor income (beta > 0):")
for j in range(n_cats):
    me = ame_matrix[0, j]  # income is first variable
    print(f"  AME on P({RATING_LABELS[j]:10s}): {me:+.4f}  "
          f"({'increases' if me > 0 else 'decreases'})")

print("\nFor debt_ratio (beta < 0):")
for j in range(n_cats):
    me = ame_matrix[1, j]  # debt_ratio is second variable
    print(f"  AME on P({RATING_LABELS[j]:10s}): {me:+.4f}  "
          f"({'increases' if me > 0 else 'decreases'})")

print("\nNote: For extreme categories, the sign matches beta.")
print("For intermediate categories, the sign may differ from beta!")
print("This is the hallmark of ordered models.")

<a id='section8'></a>

---

# Section 8: Random Effects Ordered Logit (15 min)

## 8.1 Motivation

In panel data, firms have persistent unobserved characteristics (management quality, brand reputation) that affect ratings:

$$y^*_{it} = X_{it}'\beta + \alpha_i + \varepsilon_{it}, \quad \alpha_i \sim N(0, \sigma^2_\alpha)$$

## 8.2 Estimation

The marginal likelihood integrates out $\alpha_i$ using **Gauss-Hermite quadrature**:

$$\mathcal{L}_i = \int \prod_{t=1}^{T} P(y_{it} \mid X_{it}, \alpha_i) \cdot \phi(\alpha_i) \, d\alpha_i$$

## 8.3 Key Output

- $\hat{\sigma}_\alpha$: standard deviation of the random effect
- $\hat{\rho} = \frac{\sigma^2_\alpha}{\sigma^2_\alpha + \pi^2/3}$: intraclass correlation for the latent variable

In [ ]:
# Random Effects Ordered Logit
print("Fitting Random Effects Ordered Logit (this may take a moment)...")
print("Using Gauss-Hermite quadrature with 12 points.\n")

model_re = RandomEffectsOrderedLogit(
    endog=y,
    exog=X,
    groups=groups,
    time=time,
    n_categories=4,
    quadrature_points=12
)
model_re.exog_names = exog_vars

results_re = model_re.fit(maxiter=500)

print("\n" + "=" * 70)
print(" " * 10 + "RANDOM EFFECTS ORDERED LOGIT")
print("=" * 70)

# Display results
print(f"\nNumber of obs:        {model_re.n_obs:>8d}")
print(f"Number of firms:      {model_re.n_entities:>8d}")
print(f"Number of categories: {model_re.n_categories:>8d}")
print(f"Log-likelihood:       {results_re.llf:>8.3f}")
print(f"Converged:            {results_re.converged}")

print(f"\nCoefficients:")
print(f"{'Variable':<15} {'Pooled':>10} {'RE':>10} {'Change':>10}")
print("-" * 50)
for k, var in enumerate(exog_vars):
    b_pooled = results_ologit.beta[k]
    b_re = results_re.beta[k]
    change = ((b_re - b_pooled) / abs(b_pooled)) * 100 if abs(b_pooled) > 1e-6 else np.nan
    print(f"{var:<15} {b_pooled:>10.4f} {b_re:>10.4f} {change:>+9.1f}%")

print(f"\nRandom Effects Parameters:")
sigma_alpha = results_re.sigma_alpha
rho = sigma_alpha**2 / (sigma_alpha**2 + np.pi**2 / 3)
print(f"  sigma_alpha = {sigma_alpha:.4f}")
print(f"  rho (ICC)   = {rho:.4f}")
print(f"\nInterpretation:")
print(f"  {rho:.1%} of the latent variable variance is due to firm-level heterogeneity.")
print(f"  Firms have persistent unobserved characteristics affecting their ratings.")

<a id='section9'></a>

---

# Section 9: Application — Credit Rating Analysis (40 min)

**Research Question**: What financial characteristics drive credit rating categories? How does the parallel regression assumption hold?

## 9.1 Model Comparison Table

In [ ]:
# Model comparison table
print("=" * 70)
print(" " * 15 + "MODEL COMPARISON")
print("=" * 70)

n_params_logit = K + 3
n_params_probit = K + 3
n_params_re = K + 3 + 1  # + sigma_alpha

comparison = pd.DataFrame({
    'Ordered Logit': {
        'Log-likelihood': results_ologit.llf,
        'N parameters': n_params_logit,
        'AIC': -2 * results_ologit.llf + 2 * n_params_logit,
        'BIC': -2 * results_ologit.llf + np.log(n_obs) * n_params_logit,
    },
    'Ordered Probit': {
        'Log-likelihood': results_oprobit.llf,
        'N parameters': n_params_probit,
        'AIC': -2 * results_oprobit.llf + 2 * n_params_probit,
        'BIC': -2 * results_oprobit.llf + np.log(n_obs) * n_params_probit,
    },
    'RE Ordered Logit': {
        'Log-likelihood': results_re.llf,
        'N parameters': n_params_re,
        'AIC': -2 * results_re.llf + 2 * n_params_re,
        'BIC': -2 * results_re.llf + np.log(n_obs) * n_params_re,
    }
})

print(comparison.round(2))

# Save comparison table
comparison.to_csv(TABLE_DIR / '07_model_comparison.csv')
print(f"\nTable saved to outputs/tables/07_model_comparison.csv")

In [ ]:
# Coefficient comparison visualization
fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(len(exog_vars))
width = 0.25

bars1 = ax.bar(x - width, results_ologit.beta, width,
               label='Ordered Logit', color='#3498db', alpha=0.8, edgecolor='black')
bars2 = ax.bar(x, results_oprobit.beta, width,
               label='Ordered Probit', color='#e74c3c', alpha=0.8, edgecolor='black')
bars3 = ax.bar(x + width, results_re.beta, width,
               label='RE Ordered Logit', color='#2ecc71', alpha=0.8, edgecolor='black')

ax.axhline(y=0, color='black', linewidth=0.8, alpha=0.5)
ax.set_xlabel('Variable')
ax.set_ylabel('Coefficient')
ax.set_title('Coefficient Comparison Across Ordered Models', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(exog_vars)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIG_DIR / '07_credit_coefficient_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved to outputs/figures/07_credit_coefficient_comparison.png")

## 9.2 Policy Analysis: Income and Rating Transitions

In [ ]:
# How much income improvement moves a firm from "fair" to "good"?
print("=== Policy Analysis: Income and Rating Transitions ===")

# Start with an "average" firm in the "Fair" range
base_profile = data[exog_vars].mean().values

# Vary income while holding other variables at their means
income_range = np.linspace(8, 13, 100)
probs_income = np.zeros((100, 4))

for i, inc in enumerate(income_range):
    profile = base_profile.copy()
    profile[0] = inc  # Set income
    probs_income[i] = results_ologit.predict_proba(profile.reshape(1, -1))

# Find income level where P(Good) + P(Excellent) exceeds 50%
prob_good_or_better = probs_income[:, 2] + probs_income[:, 3]
threshold_idx = np.argmax(prob_good_or_better > 0.5)
income_threshold = income_range[threshold_idx]

print(f"\nAt average characteristics:")
for j in range(4):
    base_prob = results_ologit.predict_proba(base_profile.reshape(1, -1))[0, j]
    print(f"  P({RATING_LABELS[j]}) = {base_prob:.4f}")

print(f"\nIncome threshold for P(Good or better) > 50%: {income_threshold:.2f}")
print(f"Current average income: {base_profile[0]:.2f}")
print(f"Required increase: {income_threshold - base_profile[0]:+.2f}")

In [ ]:
# Visualize probability curves as income varies
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Panel A: Individual category probabilities
ax = axes[0]
for j in range(4):
    ax.plot(income_range, probs_income[:, j], linewidth=2,
            color=RATING_COLORS[j], label=RATING_LABELS[j])

ax.axvline(x=base_profile[0], color='gray', linestyle=':', alpha=0.7, label='Mean income')
ax.set_xlabel('Log Firm Income')
ax.set_ylabel('Probability')
ax.set_title('Category Probabilities vs Income\n(other variables at means)', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel B: Cumulative probabilities
ax = axes[1]
cum_probs = np.cumsum(probs_income, axis=1)
for j in range(3):
    ax.plot(income_range, cum_probs[:, j], linewidth=2,
            color=RATING_COLORS[j], label=f'P(rating <= {j})')

ax.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
ax.axvline(x=base_profile[0], color='gray', linestyle=':', alpha=0.7)
ax.set_xlabel('Log Firm Income')
ax.set_ylabel('Cumulative Probability')
ax.set_title('Cumulative Probability Curves\nP(rating <= j) vs Income', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(-0.05, 1.05)

plt.suptitle('Credit Rating Response to Income Changes', fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIG_DIR / '07_credit_income_response.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved to outputs/figures/07_credit_income_response.png")

## 9.3 Healthy vs Distressed Firms

In [ ]:
# Compare healthy vs distressed firm profiles
print("=== Healthy vs Distressed Firm Comparison ===")

healthy = np.array([12.0, 0.15, 30, 10.0, 0.15])   # high income, low debt, large, profitable
distressed = np.array([9.0, 0.70, 10, 6.5, 0.01])   # low income, high debt, small, low profit

prob_healthy = results_ologit.predict_proba(healthy.reshape(1, -1))[0]
prob_distressed = results_ologit.predict_proba(distressed.reshape(1, -1))[0]

print(f"\n{'Category':<12} {'Healthy':>10} {'Distressed':>12} {'Difference':>12}")
print("-" * 50)
for j in range(4):
    diff = prob_healthy[j] - prob_distressed[j]
    print(f"{RATING_LABELS[j]:<12} {prob_healthy[j]:>10.4f} {prob_distressed[j]:>12.4f} {diff:>+12.4f}")

print(f"\nPredicted rating:")
print(f"  Healthy firm:    {RATING_LABELS[np.argmax(prob_healthy)]}")
print(f"  Distressed firm: {RATING_LABELS[np.argmax(prob_distressed)]}")

## 9.4 AME Table and Report

In [ ]:
# Save AME table
ame_df.to_csv(TABLE_DIR / '07_marginal_effects.csv')
print("AME table saved to outputs/tables/07_marginal_effects.csv")

# Generate HTML report
report_html = f"""<!DOCTYPE html>
<html>
<head><title>Credit Rating Analysis Report</title>
<style>body {{font-family: Arial; margin: 40px;}}
table {{border-collapse: collapse; margin: 20px 0;}}
th, td {{border: 1px solid #ddd; padding: 8px; text-align: right;}}
th {{background-color: #3498db; color: white;}}
h1 {{color: #2c3e50;}} h2 {{color: #3498db;}}</style></head>
<body>
<h1>Credit Rating Analysis Report</h1>
<p>Generated: 2026-02-17 | PanelBox Ordered Models Tutorial</p>

<h2>Dataset</h2>
<p>Panel of {data['id'].nunique()} firms over {data['year'].nunique()} years ({len(data)} observations).</p>
<p>Rating categories: Poor (0), Fair (1), Good (2), Excellent (3).</p>

<h2>Model Comparison</h2>
{comparison.to_html()}

<h2>Average Marginal Effects (Ordered Logit)</h2>
{ame_df.round(4).to_html()}

<h2>Key Findings</h2>
<ul>
<li>Income and profitability are the strongest positive determinants of better ratings.</li>
<li>Higher debt ratio significantly reduces rating quality.</li>
<li>Random effects capture significant firm-level heterogeneity (rho = {rho:.2%}).</li>
</ul>
</body></html>"""

with open(REPORT_DIR / '07_credit_rating_analysis.html', 'w') as f:
    f.write(report_html)
print("Report saved to outputs/reports/07_credit_rating_analysis.html")

<a id='exercises'></a>

---

# Exercises

---

## Exercise 1: Ordered Logit vs Multinomial Logit (Medium)

**Objective**: Compare Ordered Logit with Multinomial Logit on ordinal data.

### Task

1. Estimate a Multinomial Logit on the credit rating data (using `MultinomialLogit`)
2. Compare AIC/BIC with the Ordered Logit
3. Count the number of parameters in each model

### Questions

1. Which model is more parsimonious?
2. Does the Ordered Logit sacrifice fit for parsimony?
3. When would you prefer MNL over Ordered Logit?

In [ ]:
# Exercise 1: Your solution here

# Step 1: Estimate Multinomial Logit
# from panelbox.models.discrete.multinomial import MultinomialLogit
# model_mnl = MultinomialLogit(endog=y, exog=X, n_alternatives=4, base_alternative=0)
# results_mnl = model_mnl.fit()

# Step 2: Compare AIC/BIC
# n_params_mnl = (4 - 1) * len(exog_vars)  # (J-1) x K
# n_params_ologit = len(exog_vars) + 3      # K + (J-1)
# print(f"MNL parameters: {n_params_mnl}")
# print(f"Ordered Logit parameters: {n_params_ologit}")

# Step 3: Compare fit
# aic_mnl = -2 * results_mnl.llf + 2 * n_params_mnl
# print(f"AIC: MNL = {aic_mnl:.2f}, OLogit = {aic_logit:.2f}")

---

## Exercise 2: Cutpoints Exploration (Easy)

**Objective**: Understand how cutpoints change with category structure.

### Task

1. Estimate Ordered Logit on the full 4-category data
2. Merge categories 0 and 1 (Poor + Fair = "Low") to create 3 categories
3. Re-estimate Ordered Logit on the 3-category data
4. Compare cutpoints and coefficients

### Questions

1. How do the cutpoints change?
2. Do the $\beta$ coefficients change significantly?
3. What happens to the log-likelihood?

In [ ]:
# Exercise 2: Your solution here

# Step 1: Full model (already estimated as results_ologit)

# Step 2: Merge categories 0 and 1
# y_merged = y.copy()
# y_merged[y_merged == 1] = 0  # Merge Fair into Poor -> "Low"
# y_merged[y_merged == 2] = 1  # Good -> 1
# y_merged[y_merged == 3] = 2  # Excellent -> 2

# Step 3: Re-estimate
# model_3cat = OrderedLogit(endog=y_merged, exog=X, groups=groups, time=time, n_categories=3)
# results_3cat = model_3cat.fit()
# print(results_3cat.summary())

# Step 4: Compare
# print(f"4-cat cutpoints: {results_ologit.cutpoints}")
# print(f"3-cat cutpoints: {results_3cat.cutpoints}")

---

## Exercise 3: Brant Test Interpretation (Medium)

**Objective**: Interpret the Brant test results and understand their implications.

### Task

1. Review the Brant test results from Section 5
2. Which variables (if any) violate proportional odds?
3. For violating variables, inspect the binary logit coefficients across thresholds
4. Explain what the violation means economically

### Questions

1. If `debt_ratio` violates proportional odds, what does it mean?
2. What model would you use if proportional odds is rejected?
3. Is it possible for proportional odds to hold for some variables but not others?

In [ ]:
# Exercise 3: Your solution here

# Step 1: Review Brant test results (from Section 5)
# Look at the per-variable p-values

# Step 2: Identify violating variables
# rejected_vars = [r['Variable'] for r in brant_results if r['p_value'] < 0.05]
# print(f"Variables violating proportional odds: {rejected_vars}")

# Step 3: Compare binary logit coefficients for these variables
# For each rejected variable, plot the coefficient across thresholds

# Step 4: Economic interpretation

---

## Exercise 4: Marginal Effects Ambiguity (Medium)

**Objective**: Demonstrate that intermediate category marginal effects can change sign.

### Task

1. Calculate the marginal effect of income on P(Fair) at different income levels
2. Show that the sign can change depending on the evaluation point
3. Plot the marginal effect of income on each category as income varies

### Hint

For observation with covariates $X$:
$$\frac{\partial P(y=j)}{\partial \text{income}} = \beta_{\text{income}} \times [f(\kappa_{j-1} - X'\beta) - f(\kappa_j - X'\beta)]$$

In [ ]:
# Exercise 4: Your solution here

# Step 1: Varying income, compute ME on each category
# base = data[exog_vars].mean().values
# income_range = np.linspace(8, 13, 100)
# me_by_income = np.zeros((100, 4))
# for i, inc in enumerate(income_range):
#     profile = base.copy()
#     profile[0] = inc
#     lp = profile @ beta
#     for j in range(4):
#         z_lo = cutpoints_ext[j] - lp
#         z_hi = cutpoints_ext[j+1] - lp
#         pdf_lo = logistic_pdf(z_lo) if np.isfinite(z_lo) else 0
#         pdf_hi = logistic_pdf(z_hi) if np.isfinite(z_hi) else 0
#         me_by_income[i, j] = beta[0] * (pdf_lo - pdf_hi)

# Step 2: Plot
# fig, ax = plt.subplots(figsize=(10, 6))
# for j in range(4):
#     ax.plot(income_range, me_by_income[:, j], label=RATING_LABELS[j], color=RATING_COLORS[j])
# ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
# ax.set_title('Marginal Effect of Income on P(rating=j)')
# ax.legend()
# plt.show()

---

## Exercise 5: RE vs Pooled Ordered Logit (Hard)

**Objective**: Evaluate the importance of unobserved heterogeneity.

### Task

1. Compare the pooled and RE Ordered Logit estimates
2. Test whether $\sigma_\alpha$ is significant (LR test: compare log-likelihoods)
3. Calculate the intraclass correlation $\rho$
4. Interpret: what does $\rho$ tell us about firm-level heterogeneity?

### Hint

LR test: $\chi^2 = -2 \times (\ell_{\text{pooled}} - \ell_{\text{RE}})$, df=1, but uses a mixture distribution (boundary problem).

In [ ]:
# Exercise 5: Your solution here

# Step 1: Compare estimates (use results_ologit and results_re from above)
# for k, var in enumerate(exog_vars):
#     print(f"{var}: pooled={results_ologit.beta[k]:.4f}, RE={results_re.beta[k]:.4f}")

# Step 2: LR test
# lr_stat = -2 * (results_ologit.llf - results_re.llf)
# p_lr = 0.5 * (1 - chi2.cdf(lr_stat, 1))  # mixture chi2 for boundary
# print(f"LR statistic: {lr_stat:.4f}")
# print(f"p-value (mixture): {p_lr:.4f}")

# Step 3: Interpret rho
# sigma = results_re.sigma_alpha
# rho = sigma**2 / (sigma**2 + np.pi**2/3)
# print(f"rho = {rho:.4f}")

---

# Summary and Key Takeaways

## What We Learned

1. **Ordered models** are appropriate when the dependent variable has a natural ordering (poor < fair < good < excellent), unlike MNL which ignores order

2. **Latent variable framework**: An unobserved continuous $y^*$ is mapped to observed categories via cutpoints $\kappa_j$

3. **Coefficient interpretation**: $\beta_k > 0$ means higher $X_k$ shifts probability toward higher categories, but this is NOT the marginal effect

4. **Cutpoints** are estimated alongside $\beta$ and define flexible category boundaries

5. **Parallel regression assumption**: $\beta$ is common across all thresholds. Always test with Brant test

6. **Category-specific marginal effects**: Sum to zero across categories. Extreme categories follow the sign of $\beta$; intermediate categories are ambiguous

7. **Random Effects**: Account for firm-level persistent heterogeneity; $\rho$ measures its importance

## Key Formulas

| Concept | Formula |
|---------|--------|
| Latent variable | $y^* = X'\beta + \varepsilon$ |
| Category probability | $P(y=j) = F(\kappa_j - X'\beta) - F(\kappa_{j-1} - X'\beta)$ |
| Marginal effect | $\frac{\partial P(y=j)}{\partial x_k} = \beta_k [f(\kappa_{j-1} - X'\beta) - f(\kappa_j - X'\beta)]$ |
| Logit link | $F = \Lambda$ (logistic CDF) |
| Probit link | $F = \Phi$ (normal CDF) |
| ICC | $\rho = \sigma^2_\alpha / (\sigma^2_\alpha + \pi^2/3)$ |

## Common Pitfalls

1. Interpreting $\beta$ as marginal effects (they are not — always compute AME)
2. Ignoring intermediate category ambiguity (AME sign can differ from $\beta$)
3. Skipping the Brant test (proportional odds violation leads to inconsistency)
4. Using MNL on ordinal data (wastes information) or ordered models on nominal data (imposes wrong structure)
5. Interpreting cutpoints as "distances" between categories without considering scale

## Next Steps

- **Generalized Ordered Logit**: Relaxes proportional odds for specific variables
- **Dynamic Ordered Models**: State dependence in ratings
- **Fixed Effects Ordered Logit**: Bias-corrected estimation (Baetschmann et al., 2015)

---

## References

### Essential Reading

1. McKelvey, R. D., & Zavoina, W. (1975). A statistical model for the analysis of ordinal level dependent variables. *Journal of Mathematical Sociology*.

2. Brant, R. (1990). Assessing proportionality in the proportional odds model. *Biometrics*.

### Textbooks

3. Long, J. S., & Freese, J. (2014). *Regression Models for Categorical Dependent Variables Using Stata*. Ch. 7.

4. Wooldridge, J. M. (2010). *Econometric Analysis of Cross Section and Panel Data*. Ch. 15.

---

**End of Notebook 07: Ordered Logit/Probit Models**

You're now ready to explore advanced topics in ordered choice models or revisit earlier notebooks for comparison.